In [1]:
import pandas as pd
import sklearn
import numpy as np
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
import torch
from datasets import load_dataset
import tensorflow as tf
import tensorflow_addons as tfa
import gc

from keras import optimizers
import matplotlib.pyplot as plt
from keras import initializers
from keras.utils import np_utils
from keras import regularizers
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import concatenate
from keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization, ReLU
from keras.layers import LSTM, Bidirectional
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score


import keras

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_goss_train = pd.read_pickle('../../Dissertation - Fake News/Embeddings/df_train_xlnet_simple.pkl')
df_goss_val = pd.read_pickle('../../Dissertation - Fake News/Embeddings/df_val_xlnet_simple.pkl')
df_goss_test = pd.read_pickle('../../Dissertation - Fake News/Embeddings/df_test_xlnet_simple.pkl')

In [3]:
df_goss_train.head()

,hidden_state,label,idx
0,"[[-0.6905054, 0.10080558, -3.5896642, 0.688696...",0,1934
1,"[[0.71386486, 1.624135, 0.39983767, 1.6060241,...",0,1408
2,"[[-1.4997008, -2.5545237, -2.6850965, 2.128371...",0,2589
3,"[[-0.59582114, 0.6739678, -0.54739684, 0.43669...",1,5778
4,"[[-0.17796452, 1.5538232, -0.1271606, 0.483559...",0,2922


In [4]:
def label_change(df):
    if df['label'] == 0: return 1
    else: return 0

In [5]:
df_goss_train['target'] = df_goss_train.apply(label_change, axis=1)
df_goss_train.head()

,hidden_state,label,idx,target
0,"[[-0.6905054, 0.10080558, -3.5896642, 0.688696...",0,1934,1
1,"[[0.71386486, 1.624135, 0.39983767, 1.6060241,...",0,1408,1
2,"[[-1.4997008, -2.5545237, -2.6850965, 2.128371...",0,2589,1
3,"[[-0.59582114, 0.6739678, -0.54739684, 0.43669...",1,5778,0
4,"[[-0.17796452, 1.5538232, -0.1271606, 0.483559...",0,2922,1


In [6]:
df_goss_val['target'] = df_goss_val.apply(label_change, axis=1)
df_goss_test['target'] = df_goss_test.apply(label_change, axis=1)

In [7]:
# Count samples per class: 0-real, 1-fake
classes_zero = df_goss_train[df_goss_train['target'] == 0]
classes_one = df_goss_train[df_goss_train['target'] == 1]

# Convert parts into NumPy arrays for weight computation
zero_numpy = classes_zero['target'].to_numpy()
one_numpy = classes_one['target'].to_numpy()
all_together = np.concatenate((zero_numpy, one_numpy))
unique_classes = np.unique(all_together)

# Compute weights
weights = sklearn.utils.class_weight.compute_class_weight('balanced', unique_classes, all_together)
print(weights)

[0.64400494 2.2360515 ]


C:\Users\labca\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 1 1 1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [8]:
weights_ker = {i : weights[i] for i in range(2)}
weights_ker

{0: 0.6440049443757726, 1: 2.236051502145923}

In [9]:
array_train = df_goss_train[["hidden_state","target"]].to_numpy() 
X_train = np.array(array_train[:,0].tolist())
X_train.shape

(10420, 1, 768)

In [10]:
X_train = np.squeeze(X_train, 1)
X_train.shape

(10420, 768)

In [11]:
y_label = np.array(array_train[:,1].tolist())
y_label.shape

(10420,)

In [12]:
array_val = df_goss_val[["hidden_state","target"]].to_numpy() 
X_val = np.array(array_val[:,0].tolist())
X_val = np.squeeze(X_val, 1)
y_val_label = np.array(array_val[:,1].tolist())

array_test = df_goss_test[["hidden_state","target"]].to_numpy() 
X_test = np.array(array_test[:,0].tolist())
X_test = np.squeeze(X_test, 1)
y_test_label = np.array(array_test[:,1].tolist())

In [13]:
def reset_tensorflow_keras_backend():
    keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()
    _ = gc.collect()

In [14]:
def reinitialize(model):
    for l in model.layers:
        if hasattr(l,"kernel_initializer"):
            l.kernel.assign(l.kernel_initializer(tf.shape(l.kernel)))
        if hasattr(l,"bias_initializer"):
            l.bias.assign(l.bias_initializer(tf.shape(l.bias)))
        if hasattr(l,"recurrent_initializer"):
            l.recurrent_kernel.assign(l.recurrent_initializer(tf.shape(l.recurrent_kernel)))

In [15]:
def run_model(X_train, train_label, X_val, val_label, X_test, test_label, batch_size=32,epochs=100, verbose=1, class_weight=weights_ker, \
              itr=30, shuffle=True):
    
    adam = optimizers.Adam(learning_rate=1e-4)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    real_precision_list = []
    real_recall_list = []
    real_f1_list = []
    fake_precision_list = []
    fake_recall_list = []
    fake_f1_list = []
    
    macro_precision_list = []
    macro_recall_list = []
    macro_f1_list = []
    accuracy_list = []
    num_epochs = []

    #Define model
    input_text = Input(shape=(768))
    dense_text = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01),\
                kernel_initializer=initializers.he_normal(seed=0))(input_text)
    dropout = Dropout(0.4)(dense_text)
    dense_text = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01),\
                kernel_initializer=initializers.he_normal(seed=0))(dropout)
    dense_text = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01),\
                kernel_initializer=initializers.he_normal(seed=0))(dense_text)
    output = Dense(1, activation='sigmoid')(dense_text)
    model = Model(inputs=input_text, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics='accuracy')
        
    for i in range(0,itr):
        print("\nTraining for iteration number: ", i)
        reset_tensorflow_keras_backend()
        reinitialize(model)
        history = model.fit(X_train, train_label, validation_data=(X_val, val_label), batch_size=batch_size,epochs=epochs,
                            callbacks=[callback], verbose=verbose, class_weight=class_weight, shuffle=shuffle)
        
        y_pred = model.predict(X_test)
        predictions = list(map(lambda x: 0 if x<0.5 else 1, y_pred))
        
        report = classification_report(y_test_label, predictions, target_names=['real','fake'], output_dict=True)
        real_precision = report['real']['precision'] 
        real_recall = report['real']['recall'] 
        real_f1 = report['real']['f1-score'] 
        fake_precision = report['fake']['precision'] 
        fake_recall = report['fake']['recall'] 
        fake_f1 = report['fake']['f1-score'] 
        
        macro_precision =  report['macro avg']['precision'] 
        macro_recall = report['macro avg']['recall']    
        macro_f1 = report['macro avg']['f1-score']
        accuracy = report['accuracy']

        real_precision_list.append(real_precision)
        real_recall_list.append(real_recall)
        real_f1_list.append(real_f1)
        fake_precision_list.append(fake_precision)
        fake_recall_list.append(fake_recall)
        fake_f1_list.append(fake_f1)
        
        macro_precision_list.append(macro_precision)
        macro_recall_list.append(macro_recall)
        macro_f1_list.append(macro_f1)
        accuracy_list.append(accuracy)
        num_epochs.append(len(history.history['loss']))
        
        
    mean_real_precision = np.mean(real_precision_list)
    mean_real_recall = np.mean(real_recall_list)
    mean_real_f1 = np.mean(real_f1_list)
    mean_fake_precision = np.mean(fake_precision_list)
    mean_fake_recall= np.mean(fake_recall_list)
    mean_fake_f1 = np.mean(fake_f1_list)
    
    mean_macro_precision = np.mean(macro_precision_list)
    mean_macro_recall = np.mean(macro_recall)
    mean_macro_f1 = np.mean(macro_f1_list)
    mean_accuracy = np.mean(accuracy_list)
    mean_epoch = np.mean(num_epochs)
    
    std_real_precision = np.std(real_precision_list)
    std_real_recall = np.std(real_recall_list)
    std_real_f1 = np.std(real_f1_list)
    std_fake_precision = np.std(fake_precision_list)
    std_fake_recall= np.std(fake_recall_list)
    std_fake_f1 = np.std(fake_f1_list)
    
    std_macro_precision = np.std(macro_precision_list)
    std_macro_recall = np.std(macro_recall)
    std_macro_f1 = np.std(macro_f1_list)
    std_accuracy = np.std(accuracy_list)

    print (report)
    
    print("The standard deviation on real precision : ", std_real_precision)
    print("The standard deviation on real recall: ", std_real_recall)
    print("The standard deviation on real f1: ", std_real_f1)
    print("The standard deviation on fake precision : ", std_fake_precision)
    print("The standard deviation on fake recall: ", std_fake_recall)
    print("The standard deviation on macro precision: ", std_macro_precision)
    print("The standard deviation on macro recall: ", std_macro_recall)
    print("The standard deviation on macro_f1: ", std_macro_f1)
    print("The standard deviation on macro accuracy: ", std_accuracy)
    print("Average number of epochs", mean_epoch)
    
    return real_precision_list, real_recall_list, real_f1_list, fake_precision_list, fake_recall_list, fake_f1_list,\
    macro_precision_list, macro_recall_list, macro_f1_list, accuracy_list

In [16]:
real_precision, real_recall, real_f_score, fake_precision, fake_recall,\
fake_f1, macro_precision, macro_recall, macro_f1, accuracy = run_model(X_train, \
y_label, X_val, y_val_label, X_test, y_test_label, verbose=0, itr=30)


Training for iteration number:  0
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  1
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  2
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  3
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  4
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  5
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  6
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  7
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  8
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  9
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  10
41/41 [==============================] - 0s 1ms/step

Training for iteration number:

In [17]:
print("Fake Precision:\n ", fake_precision)
print("Fake Recall:\n ", fake_recall)
print("Fake F1-Score:\n ", fake_f1)

Fake Precision:
  [0.6477732793522267, 0.6111111111111112, 0.6066666666666667, 0.5791044776119403, 0.6612244897959184, 0.553314121037464, 0.6875, 0.6346863468634686, 0.5932203389830508, 0.7348066298342542, 0.5840978593272171, 0.6006711409395973, 0.6328671328671329, 0.6200716845878136, 0.6091549295774648, 0.5849056603773585, 0.607773851590106, 0.6064516129032258, 0.6199261992619927, 0.5953177257525084, 0.5271317829457365, 0.6213235294117647, 0.7035175879396985, 0.5429362880886427, 0.5815384615384616, 0.5853658536585366, 0.5462184873949579, 0.6164874551971327, 0.5918367346938775, 0.696969696969697]
Fake Recall:
  [0.5633802816901409, 0.6584507042253521, 0.6408450704225352, 0.6830985915492958, 0.5704225352112676, 0.676056338028169, 0.5035211267605634, 0.6056338028169014, 0.6161971830985915, 0.46830985915492956, 0.6725352112676056, 0.6302816901408451, 0.6373239436619719, 0.6091549295774648, 0.6091549295774648, 0.6549295774647887, 0.6056338028169014, 0.6619718309859155, 0.5915492957746479, 

In [20]:
print("Real Precision:\n ", real_precision)
print("Real Recall:\n ", real_recall)
print("Real F1-Score:\n ",  real_f_score) 

Real Precision:
  [0.8825757575757576, 0.9027081243731193, 0.8983050847457628, 0.9070247933884298, 0.8846880907372401, 0.9037656903765691, 0.8712328767123287, 0.8914728682170543, 0.8918650793650794, 0.8654188948306596, 0.9047131147540983, 0.8955223880597015, 0.8987217305801377, 0.8916015625, 0.8910696761530913, 0.9005076142131979, 0.8901960784313725, 0.9033232628398792, 0.8875968992248062, 0.8944223107569721, 0.9126637554585153, 0.8884578079534433, 0.8695652173913043, 0.9065817409766455, 0.9028629856850716, 0.9056410256410257, 0.9059196617336153, 0.890625, 0.8909811694747275, 0.8678733031674208]
Real Recall:
  [0.914622178606477, 0.8832188420019627, 0.8842001962708538, 0.8616290480863592, 0.9185475956820413, 0.8478900883218842, 0.9362119725220804, 0.9028459273797841, 0.8822374877330716, 0.9528949950932286, 0.8665358194308145, 0.8832188420019627, 0.8969578017664377, 0.8959764474975466, 0.8910696761530913, 0.8704612365063789, 0.8910696761530913, 0.8802747791952895, 0.8989205103042198, 0.

In [18]:
print("Macro Precision:\n ", macro_precision)
print("Macro Recall:\n ", macro_recall)
print("Macro F1-Score:\n ", macro_f1)
print("Accuracy Score:\n ", accuracy)

Macro Precision:
  [0.7651745184639922, 0.7569096177421153, 0.7524858757062147, 0.7430646355001851, 0.7729562902665792, 0.7285399057070165, 0.7793664383561644, 0.7630796075402615, 0.7425427091740651, 0.8001127623324569, 0.7444054870406578, 0.7480967644996495, 0.7657944317236354, 0.7558366235439068, 0.7501123028652781, 0.7427066372952782, 0.7489849650107392, 0.7548874378715524, 0.7537615492433994, 0.7448700182547403, 0.7198977692021259, 0.754890668682604, 0.7865414026655014, 0.7247590145326441, 0.7422007236117666, 0.745503439649781, 0.7260690745642866, 0.7535562275985663, 0.7414089520843026, 0.7824215000685589]
Macro Recall:
  [0.7390012301483089, 0.7708347731136573, 0.7625226333466946, 0.7723638198178275, 0.7444850654466544, 0.7619732131750265, 0.7198665496413219, 0.7542398650983427, 0.7492173354158316, 0.7106024271240791, 0.76953551534921, 0.7567502660714038, 0.7671408727142048, 0.7525656885375056, 0.7501123028652781, 0.7626954069855838, 0.7483517394849963, 0.7711233050906026, 0.74523

In [1]:
import numpy as np
acc_mean = np.mean([0.8380660015349194, 0.8342287029930928, 0.8311588641596316, 0.8227168073676132, 0.8426707597851113, 0.8104374520337683, 0.841903300076746, 0.8380660015349194, 0.8242517267843438, 0.8472755180353031, 0.8242517267843438, 0.8280890253261703, 0.8403683806600154, 0.8334612432847276, 0.829623944742901, 0.8234842670759785, 0.8288564850345357, 0.8326937835763623, 0.8319263238679969, 0.8257866462010744, 0.7981580966999232, 0.8326937835763623, 0.8442056792018419, 0.8058326937835764, 0.8227168073676132, 0.8250191864927091, 0.807367613200307, 0.8319263238679969, 0.8234842670759785, 0.841903300076746])
acc_mean

0.8287541570734204

In [2]:
real_precision_mean = np.mean([0.8825757575757576, 0.9027081243731193, 0.8983050847457628, 0.9070247933884298, 0.8846880907372401, 0.9037656903765691, 0.8712328767123287, 0.8914728682170543, 0.8918650793650794, 0.8654188948306596, 0.9047131147540983, 0.8955223880597015, 0.8987217305801377, 0.8916015625, 0.8910696761530913, 0.9005076142131979, 0.8901960784313725, 0.9033232628398792, 0.8875968992248062, 0.8944223107569721, 0.9126637554585153, 0.8884578079534433, 0.8695652173913043, 0.9065817409766455, 0.9028629856850716, 0.9056410256410257, 0.9059196617336153, 0.890625, 0.8909811694747275, 0.8678733031674208])
real_precision_mean

0.8932634521772342

In [3]:
real_recall_mean = np.mean([0.914622178606477, 0.8832188420019627, 0.8842001962708538, 0.8616290480863592, 0.9185475956820413, 0.8478900883218842, 0.9362119725220804, 0.9028459273797841, 0.8822374877330716, 0.9528949950932286, 0.8665358194308145, 0.8832188420019627, 0.8969578017664377, 0.8959764474975466, 0.8910696761530913, 0.8704612365063789, 0.8910696761530913, 0.8802747791952895, 0.8989205103042198, 0.8812561334641805, 0.8204121687929342, 0.8989205103042198, 0.9421000981354269, 0.8380765456329735, 0.8665358194308145, 0.8665358194308145, 0.8410206084396468, 0.8949950932286556, 0.8822374877330716, 0.9411187438665358])
real_recall_mean

0.8877330716388616

In [4]:
real_f1_mean = np.mean([0.8983132530120482, 0.8928571428571428, 0.8911968348170127, 0.8837443381982889, 0.9012999518536351, 0.8749367088607595, 0.9025543992431408, 0.8971233544612385, 0.8870251603354711, 0.9070527790751984, 0.8852130325814536, 0.8893280632411067, 0.8978388998035363, 0.8937836514929025, 0.8910696761530913, 0.8852295409181635, 0.8906326630701324, 0.8916500994035785, 0.8932228181374938, 0.8877904102817598, 0.8640826873385012, 0.8936585365853659, 0.9043805934997644, 0.8709841917389087, 0.8843264897346018, 0.8856569709127383, 0.872264631043257, 0.8928046989720998, 0.8865877712031558, 0.9030131826741996])
real_f1_mean

0.8899874177166581

In [5]:
fake_precision_mean = np.mean([0.6477732793522267, 0.6111111111111112, 0.6066666666666667, 0.5791044776119403, 0.6612244897959184, 0.553314121037464, 0.6875, 0.6346863468634686, 0.5932203389830508, 0.7348066298342542, 0.5840978593272171, 0.6006711409395973, 0.6328671328671329, 0.6200716845878136, 0.6091549295774648, 0.5849056603773585, 0.607773851590106, 0.6064516129032258, 0.6199261992619927, 0.5953177257525084, 0.5271317829457365, 0.6213235294117647, 0.7035175879396985, 0.5429362880886427, 0.5815384615384616, 0.5853658536585366, 0.5462184873949579, 0.6164874551971327, 0.5918367346938775, 0.696969696969697])
fake_precision_mean

0.6127990378759675

In [6]:
fake_recall_mean = np.mean([0.5633802816901409, 0.6584507042253521, 0.6408450704225352, 0.6830985915492958, 0.5704225352112676, 0.676056338028169, 0.5035211267605634, 0.6056338028169014, 0.6161971830985915, 0.46830985915492956, 0.6725352112676056, 0.6302816901408451, 0.6373239436619719, 0.6091549295774648, 0.6091549295774648, 0.6549295774647887, 0.6056338028169014, 0.6619718309859155, 0.5915492957746479, 0.6267605633802817, 0.7183098591549296, 0.5950704225352113, 0.49295774647887325, 0.6901408450704225, 0.6654929577464789, 0.676056338028169, 0.6866197183098591, 0.6056338028169014, 0.6126760563380281, 0.4859154929577465])
fake_recall_mean

0.6171361502347419

In [7]:
fake_f1_mean = np.mean( [0.6026365348399247, 0.6338983050847459, 0.6232876712328768, 0.6268174474959612, 0.612476370510397, 0.6085578446909669, 0.58130081300813, 0.6198198198198198, 0.6044905008635578, 0.5720430107526883, 0.6252045826513911, 0.6151202749140893, 0.6350877192982456, 0.6145648312611012, 0.6091549295774648, 0.6179401993355482, 0.6067019400352733, 0.632996632996633, 0.6054054054054053, 0.6106346483704975, 0.6080476900149032, 0.6079136690647482, 0.5797101449275363, 0.607751937984496, 0.6206896551724138, 0.6274509803921567, 0.6084243369734789, 0.61101243339254, 0.6020761245674741, 0.5726141078838175])
fake_f1_mean

0.610127685417276

In [2]:
macro_f1_mean = np.mean([0.7504748939259864, 0.7633777239709443, 0.7572422530249447, 0.7552808928471251, 0.7568881611820161, 0.7417472767758632, 0.7419276061256355, 0.7584715871405292, 0.7457578305995145, 0.7395478949139433, 0.7552088076164223, 0.7522241690775979, 0.7664633095508909, 0.7541742413770018, 0.7501123028652781, 0.7515848701268559, 0.7486673015527029, 0.7623233662001058, 0.7493141117714496, 0.7492125293261287, 0.7360651886767022, 0.7507861028250571, 0.7420453692136504, 0.7393680648617024, 0.7525080724535078, 0.7565539756524475, 0.7403444840083679, 0.7519085661823199, 0.744331947885315, 0.7378136452790085])
macro_f1_mean

0.7500575515669671